
# Cambios Recientes en los Códigos Postales de México (2020–2023)

## Actualizaciones a Nivel Nacional por Año

1. **Cambios en 2020:** No se encontró un número oficial públicamente divulgado sobre cuántos códigos postales se modificaron en 2020. Sin embargo, los datos disponibles sugieren que fue un año de pocos cambios netos en comparación con años posteriores. Dado que 2022 fue destacado como un año **récord** en cambios (ver abajo), se infiere que 2020 tuvo **considerablemente menos** modificaciones (probablemente solo algunos cientos de nuevos códigos) a nivel nacional. *Esto coincide con la relativa desaceleración en desarrollos urbanos durante la etapa inicial de la pandemia de COVID-19, lo cual habría limitado la creación de nuevos códigos postales en 2020.* (No hay una cifra exacta publicada en las fuentes conectadas para este año.)

2. **Cambios en 2022:** Fue el año con **mayor número de modificaciones de códigos postales** en México en tiempos recientes. De acuerdo con un análisis de GeoPostcodes, en 2022 se **introdujeron o cambiaron 837 códigos postales** a nivel nacional. Este pico refleja el **dinámico crecimiento urbano** y ajustes administrativos en distintas regiones del país durante ese año. En otras palabras, **837 nuevas zonas** recibieron códigos postales (o se actualizaron códigos existentes) solo en 2022, marcando un récord en la década reciente.

3. **Cambios en 2023:** Tampoco existe una cifra oficial publicada para 2023, pero las tendencias indican que el número de modificaciones fue **menor que el de 2022** (posiblemente en el rango de algunos pocos cientos de nuevos códigos). GeoPostcodes destacó únicamente a 2022 como el año de mayor actualización, lo que implica que **2023 no superó ese nivel**. Para tener contexto, el número total de códigos postales en México aumentó de alrededor de **\~33 mil** a inicios de la década a **más de 36 mil** hacia 2025, crecimiento que incluye el gran salto de 2022. Es razonable asumir que 2023 contribuyó con cierta porción de ese incremento (aunque menor que 2022). Las evidencias de solicitudes de nuevos códigos en 2023 corroboran que siguieron asignándose códigos en zonas de desarrollo, pero sin alcanzar la magnitud excepcional del año previo.

## Enfoque en la Zona del Valle de México (CDMX y Edomex)

La zona metropolitana del Valle de México —que abarca la **Ciudad de México** (CDMX) y el **Estado de México** (Edomex)— concentra una porción importante de los códigos postales del país. Actualmente, la Ciudad de México cuenta con **1,102 códigos postales**, mientras que el Estado de México posee **2,354 códigos**. Esto suma más de 3,400 códigos postales solo en la región metropolitana, reflejando su alta densidad de colonias y localidades.

Dado el intenso desarrollo urbano en el Valle de México, **muchos de los cambios de códigos postales ocurridos en 2020–2023 se concentraron en esta zona**. Por ejemplo, en 2023 se documentó una modificación en la Alcaldía **Xochimilco (CDMX)**, donde el código postal **16730 fue cambiado por el 13093** para actualizar la división postal de esa área. Asimismo, en octubre de 2023 se asignó un **nuevo código postal** para la zona de *Lomas de Tecamachalco* en el Estado de México, a solicitud de autoridades federales, con el fin de reconocer un inmueble y sector específico dentro de ese fraccionamiento. Estos casos ilustran cómo las **colonias en expansión o recién regularizadas** en la metrópoli recibieron códigos postales nuevos o ajustados en los años recientes.

En resumen, **2022 fue el año con más cambios** (837 nuevos códigos a nivel nacional), mientras que **2020 y 2023 tuvieron menos modificaciones** cada uno. Gran parte de estos cambios —especialmente las asignaciones de nuevos códigos— ocurrieron en zonas de crecimiento urbano como la **Ciudad de México y el Estado de México**, asegurando que la infraestructura postal se mantenga al día con la expansión de la mancha urbana en el Valle de México. Las cifras reflejan la importancia de mantener actualizado el *Catálogo Nacional de Códigos Postales* para **servir adecuadamente a las áreas urbanas en constante evolución**, evitando confusiones en la logística de correos y paquetería.

**Fuentes:** GeoPostcodes (actualizaciones globales de códigos postales, 2024); Portal oficial *codigo-postal.co* (estadísticas de códigos por estado); Solicitudes de asignación de CP (Transparencia SEPOMEX, 2023).


In [1]:
import importlib
import numpy as np
import load_Data
import pandas as pd
import geopandas as gpd
importlib.reload(load_Data)
from load_Data import LoadData
from fuzzywuzzy import fuzz
import pickle
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from collections import Counter
from typing import Dict, Tuple
import unicodedata
import re
import os
from datetime import datetime
import pyarrow
import fastparquet
import time
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor, as_completed
from functools import partial
from scipy.spatial.distance import cdist
import recordlinkage
from recordlinkage.preprocessing import clean
from sklearn.neighbors import BallTree
import numpy as np
from geopy.distance import geodesic







In [2]:
loader = LoadData("../")

Cargamos los diccionarios de dataframes y los dataframes de GeoPostcodes y código postal para su uso posterior.



In [3]:
# Cargar datasets originales
original_datasets = loader.load_original_datasets()

# Cargar datasets clasificados donde los códigos postales son nulos
null_datasets = loader.load_classified_null_datasets()
equal_datasets = loader.load_classified_equal_datasets()
diff_datasets = loader.load_classified_different_datasets()

# Cargar datasets procesados
processed_datasets = loader.load_processed_datasets()

# Cargar GeoDataFrame con índice de marginación
marginacion_gdf = loader.load_marginacion_gdf(as_dataframe=False)

# Usar como DataFrame regular (sin geometrías)
marginacion_df = loader.load_marginacion_gdf(as_dataframe=True)

# Cargar catálogo de códigos postales
catalogo_cp = loader.load_postal_codes_catalog()

Generamos una homegindad en el nombre los campos que nos pueden llegar a ser de utilidad

In [4]:
def rename_dataframe_columns(dataframes_dict):
    
    for name, df in dataframes_dict.items():
        df_renamed = df.rename(columns={
            'SUS_CP': 'CP',
            'postal_code_api': 'CP_INDEX',
            'SUS_COL': 'COLONIA',
            'SUS_DEL': 'MUNICIPIO',
            'NOM_ENT': 'ENTIDAD',
        })
        dataframes_dict[name] = df_renamed
    
    return dataframes_dict

In [5]:
# Aplicar renombramiento a todos los diccionarios
original_datasets = rename_dataframe_columns(original_datasets)
null_datasets = rename_dataframe_columns(null_datasets)
equal_datasets = rename_dataframe_columns(equal_datasets)
diff_datasets = rename_dataframe_columns(diff_datasets)


In [6]:
for dataset_name, df in processed_datasets.items():
     df_renamed = df.rename(columns={
            'SUS_CP': 'CP',
            'postal_code_catalogo': 'CP_INDEX',
            'SUS_COL': 'COLONIA',
            'SUS_DEL': 'MUNICIPIO',
            'NOM_ENT': 'ENTIDAD',
        })
     processed_datasets[dataset_name] = df_renamed

In [7]:
catalogo_cp = catalogo_cp.rename(columns={
    'd_codigo': 'CP',
    'd_asenta': 'COLONIA',
    'D_mnpio': 'MUNICIPIO',
    'd_estado': 'ENTIDAD',
})
    

In [8]:
marginacion_gdf = marginacion_gdf.rename(columns={
    'NOM_ENT': 'ENTIDAD',
    'NOM_MUN': 'MUNICIPIO'
})

In [9]:
"""
empty_cp_datasets = {}
nonempty_cp_datasets = {}

for name, df in processed_datasets.items():
    # Creamos una máscara para NaN o strings vacíos
    mask_empty = df['CP_INDEX'].isna() | (df['CP_INDEX'].astype(str).str.strip() == '')
    # DataFrames con CP_INDEX vacío
    empty_cp_datasets[name] = df[mask_empty].copy()
    # DataFrames con CP_INDEX no vacío
    nonempty_cp_datasets[name] = df[~mask_empty].copy()
"""

"\nempty_cp_datasets = {}\nnonempty_cp_datasets = {}\n\nfor name, df in processed_datasets.items():\n    # Creamos una máscara para NaN o strings vacíos\n    mask_empty = df['CP_INDEX'].isna() | (df['CP_INDEX'].astype(str).str.strip() == '')\n    # DataFrames con CP_INDEX vacío\n    empty_cp_datasets[name] = df[mask_empty].copy()\n    # DataFrames con CP_INDEX no vacío\n    nonempty_cp_datasets[name] = df[~mask_empty].copy()\n"

In [10]:
"""
concatenated_equal = {
    year: pd.concat([equal_datasets[f"df_coordenadas_sustentantes_{year}_iguales"],
                     nonempty_cp_datasets[f"df_coordenadas_sustentantes_{year}_completos"]],
                    ignore_index=True)
    for year in [ "2020", "2021", "2022", "2023" ]
}

concatenated_null = {
    year: pd.concat([null_datasets[f"df_coordenadas_sustentantes_{year}_nulos"],
                     empty_cp_datasets[f"df_coordenadas_sustentantes_{year}_completos"]],
                    ignore_index=True)
    for year in [ "2020", "2021", "2022", "2023" ]
}
"""

'\nconcatenated_equal = {\n    year: pd.concat([equal_datasets[f"df_coordenadas_sustentantes_{year}_iguales"],\n                     nonempty_cp_datasets[f"df_coordenadas_sustentantes_{year}_completos"]],\n                    ignore_index=True)\n    for year in [ "2020", "2021", "2022", "2023" ]\n}\n\nconcatenated_null = {\n    year: pd.concat([null_datasets[f"df_coordenadas_sustentantes_{year}_nulos"],\n                     empty_cp_datasets[f"df_coordenadas_sustentantes_{year}_completos"]],\n                    ignore_index=True)\n    for year in [ "2020", "2021", "2022", "2023" ]\n}\n'

In [11]:
# los valores de marginacion_gdf tienen un .0 al final, lo cual no es deseado
# se los quitamos y buscamos que el CP tenga 5 dígitos de forma estandarizada
# convierte en marginacion_gdf 
marginacion_gdf['CP'] = (
    marginacion_gdf['CP']
    .astype(int)
    .astype(str)
    .str.zfill(5)
)




In [12]:

def clean_and_pad(df):
    #  CP: convertir a int → str
    df['CP'] = df['CP'].astype(int).astype(str)
    #  CP_INDEX: 
    #     a) convertir a str
    #     b) quitar “.0” al final
    df['CP_INDEX'] = (
        df['CP_INDEX']
        .fillna('')  # → '' en vez de NaN
        .astype(str)
        .str.replace(r'\.0$', '', regex=True)# limpia ".0"
    )
    
    # aplicar padding de un '0' a la izquierda solo cuando la longitud es 4
    for col in ['CP','CP_INDEX']:
        mask = df[col].str.len() == 4
        df.loc[mask, col] = '0' + df.loc[mask, col]
    
    return df

# aplicarlo a todos tus dataframes
for key, df in original_datasets.items():
    original_datasets[key] = clean_and_pad(df)




In [13]:
for dataset_name, df in original_datasets.items():
    print(f"Dataset: {dataset_name}")
    print(df.head())

Dataset: df_coordenadas_sustentantes_2020
                         COLONIA     CP       MUNICIPIO ENTIDAD  lat_centro  \
0  ampliacion general jose vicen  57710  nezahualcoyotl  edomex   19.384366   
1                 santa catarina  56030        chiautla  edomex   19.549580   
2              nueva san antonio  56605          chalco  edomex   19.277787   
3                       caltenco  54665       coyotepec  edomex   19.785197   
4                     hueypoxtla  55670      hueypoxtla  edomex   19.910035   

   lng_centro  lat_principal  lng_principal CP_INDEX  
0  -99.009547      19.383591     -99.010130    57710  
1  -98.880199      19.549059     -98.883151    56030  
2  -98.892980      19.278850     -98.892805    56605  
3  -99.199699      19.785052     -99.202366           
4  -99.076913      19.910934     -99.075907    55670  
Dataset: df_coordenadas_sustentantes_2021
                          COLONIA     CP       MUNICIPIO ENTIDAD  lat_centro  \
0               torres de potre

In [14]:
# Función para normalizar espacios en columnas de texto
def normalizar_espacios(df, columnas):
    """
    Normaliza espacios en las columnas especificadas:
    - Convierte múltiples espacios, tabulaciones y saltos de línea en un solo espacio
    - Elimina espacios al inicio y final del texto
    """
    for col in columnas:
        if col in df.columns:
            # Primero verificamos que sea tipo string/object
            if df[col].dtype == 'object':
                # Reemplazamos cualquier secuencia de espacios con un solo espacio
                df[col] = df[col].str.replace(r'\s+', ' ', regex=True)
                # Eliminamos espacios al inicio y final
                df[col] = df[col].str.strip()
    return df



In [15]:
# Aplicar la función a marginacion_gdf
marginacion_gdf = normalizar_espacios(marginacion_gdf, ['MUNICIPIO', 'COLONIA'])

In [16]:
for key, df in original_datasets.items():
    original_datasets[key] = normalizar_espacios(df, ['MUNICIPIO', 'COLONIA'])

In [17]:
"""
Agregamos columnas adicionales a los DataFrames originales que serán utilizadas en el proceso de llenado
  """
for key, df in original_datasets.items():
    # Añadir columna IM_2020 con valores nulos
    df['IM_2020'] = ""
    # Añadir columna HASH_CP con strings vacíos
    df['HASH_CP_gpkg'] = ""

    df['IM_DISTANCE'] = ""
    # Añadir columna Distance con valores nulos
    df['Distance'] = ""
    
    df['HASH_CP_Catalogo'] = ""
    
    df['TIPO_ASENTAMIENTO'] = ""
    
    df['TIPO_DE_ZONA']= ""
    
    
    # Actualizar el DataFrame en el diccionario
    original_datasets[key] = df

In [18]:
marginacion_gdf['latitud_centro']=""
marginacion_gdf['longitud_centro']=""

In [19]:
for dataset_name, df in original_datasets.items():
    print(f"Dataset: {dataset_name}")
    print(df.head(15))

Dataset: df_coordenadas_sustentantes_2020
                          COLONIA     CP              MUNICIPIO ENTIDAD  \
0   ampliacion general jose vicen  57710         nezahualcoyotl  edomex   
1                  santa catarina  56030               chiautla  edomex   
2               nueva san antonio  56605                 chalco  edomex   
3                        caltenco  54665              coyotepec  edomex   
4                      hueypoxtla  55670             hueypoxtla  edomex   
5               san pedro atzompa  55770                tecamac    cdmx   
6         santa ana tlachiahualpa  55994            temascalapa  edomex   
7            santa maria cozotlan  55810            teotihuacan    cdmx   
8   jardines de los claustros iii  54920              tultitlan  edomex   
9                        acatepec  43050                huautla    cdmx   
10                    santiaguito  56217                texcoco  edomex   
11                 la providencia  54783             teolo

In [20]:

def guardar_original_datasets(agregar_timestamp=True):
    """
    Guarda los DataFrames del diccionario original_datasets en formato pickle y parquet
    
    Args:
        agregar_timestamp (bool): Si True, agrega timestamp a los nombres de archivos
        
    De esta forma evitamos sobreescribir archivos existentes y podemos llevar un control de versiones sin usar csv que es más pesado y lento, y modifican los formatos de los datos.    
    """
    # Rutas base
    pickle_dir = "/home/cesar_r/Documentos/Proyectos/IberoSocialData/comipems-inequality-clustering-mapping/Match/VersionesDataFrames/pickle"
    parquet_dir = "/home/cesar_r/Documentos/Proyectos/IberoSocialData/comipems-inequality-clustering-mapping/Match/VersionesDataFrames/parquet"
    
    # Crear directorios si no existen
    os.makedirs(pickle_dir, exist_ok=True)
    os.makedirs(parquet_dir, exist_ok=True)
    
    # Generar timestamp si se solicita
    timestamp = ""
    if agregar_timestamp:
        timestamp = f"_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    print("=== Guardando original_datasets ===")
    print(f"Pickle: {pickle_dir}")
    print(f"Parquet: {parquet_dir}")
    print(f"Timestamp: {timestamp if timestamp else 'No'}")
    print("-" * 50)
    
    # Guardar cada DataFrame del diccionario original_datasets
    for dataset_name, df in original_datasets.items():
        try:
            # Guardar como S
            pickle_filepath = os.path.join(pickle_dir, f"{dataset_name}{timestamp}.pkl")
            with open(pickle_filepath, 'wb') as f:
                pickle.dump(df, f)
            print(f"✓ Pickle guardado: {dataset_name}{timestamp}.pkl")
            
            # Guardar como parquet
            parquet_filepath = os.path.join(parquet_dir, f"{dataset_name}{timestamp}.parquet")
            df.to_parquet(parquet_filepath, index=False)
            print(f"✓ Parquet guardado: {dataset_name}{timestamp}.parquet")
            
        except Exception as e:
            print(f"✗ Error guardando {dataset_name}: {str(e)}")
    
    print("-" * 50)
    print(f"Proceso completado. {len(original_datasets)} DataFrames guardados en ambos formatos.")
    
    # Guardar también el diccionario completo como pickle
    dict_pickle_path = os.path.join(pickle_dir, f"original_datasets_dict{timestamp}.pkl")
    with open(dict_pickle_path, 'wb') as f:
        pickle.dump(original_datasets, f)
    print(f"✓ Diccionario completo guardado: original_datasets_dict{timestamp}.pkl")
    
    return {
        'pickle_dir': pickle_dir,
        'parquet_dir': parquet_dir,
        'archivos_guardados': len(original_datasets)
    }

In [21]:
def guardar_original_datasets_csv(agregar_timestamp=True):
    """
    Guarda los DataFrames del diccionario original_datasets en formato CSV
    
    Args:
        agregar_timestamp (bool): Si True, agrega timestamp a los nombres de archivos
    """
    # Ruta base
    csv_dir = "/home/cesar_r/Documentos/Proyectos/IberoSocialData/comipems-inequality-clustering-mapping/Match/VersionesDataFrames/csv"
    
    # Crear directorio si no existe
    os.makedirs(csv_dir, exist_ok=True)
    
    # Generar timestamp si se solicita
    timestamp = ""
    if agregar_timestamp:
        timestamp = f"_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    print("=== Guardando original_datasets en CSV ===")
    print(f"Directorio: {csv_dir}")
    print(f"Timestamp: {timestamp if timestamp else 'No'}")
    print("-" * 50)
    
    archivos_guardados = []
    
    # Guardar cada DataFrame del diccionario original_datasets
    for dataset_name, df in original_datasets.items():
        try:
            # Guardar como CSV
            csv_filepath = os.path.join(csv_dir, f"{dataset_name}{timestamp}.csv")
            df.to_csv(csv_filepath, index=False, encoding='utf-8')
            archivos_guardados.append(csv_filepath)
            print(f"✓ CSV guardado: {dataset_name}{timestamp}.csv")
            
        except Exception as e:
            print(f"✗ Error guardando {dataset_name}: {str(e)}")
    
    print("-" * 50)
    print(f"Proceso completado. {len(archivos_guardados)} DataFrames guardados en formato CSV.")
    
    return {
        'csv_dir': csv_dir,
        'archivos_guardados': len(archivos_guardados),
        'rutas_archivos': archivos_guardados
    }



In [22]:
#resultado_csv = guardar_original_datasets_csv(agregar_timestamp=True)

In [23]:
#resultado = guardar_original_datasets(agregar_timestamp=True)

In [24]:
#resultado_csv = guardar_original_datasets_csv(agregar_timestamp=True)

In [25]:
#resultado2 = guardar_original_datasets(agregar_timestamp=True)

In [26]:

# Instancia global del cache
# Estas funciones estan DataManager pues mi primer enfoque fue corregir las entidades usando geocading con openstrreet map pero por la cantidad de instancias que se generan y la lentitud de las peticiones, mejor lo dejamos como una instancia global comentada jajaj
#geo_cache = GeocodingCache()

In [27]:
def asignar_entidad_por_codigo_postal(df):
    """
    Asigna valores y corregir valores campo 'ENTIDAD' basado en los códigos postales de CP y CP_INDEX
    
    Reglas:
    - 01000-17000: 'cdmx'
    - 50000-58000: 'edomex'
    - Otros (≥4 cifras): 'foraneo'
    
    Lógica:
    1. Si CP_INDEX está vacío/nulo → usar solo CP
    2. Si ambos existen y son iguales → usar la etiqueta correspondiente
    3. Si ambos existen pero son diferentes → comparar etiquetas:
       - Si las etiquetas son iguales → asignar esa etiqueta
       - Si las etiquetas son diferentes → no modificar (mantener valor actual)
    la idea es si los valores que no aparecen en el catálogo de códigos postales, se les asigna la etiqueta de foraneo, podemos entonces descartarlos de la base de datos para evitar errores en el llenado de los datos.
    """
    
    def obtener_etiqueta_cp(codigo_postal):
        """
        Retorna la etiqueta correspondiente al código postal
        """
        # Convertir a string y limpiar
        cp_str = str(codigo_postal).strip()
        
        # Verificar que sea un código postal válido (≥4 cifras)
        if not cp_str.isdigit() or len(cp_str) < 4:
            return None
        
        # Convertir a entero para comparación
        cp_int = int(cp_str)
        
        # Asignar etiqueta según rangos
        if 1000 <= cp_int <= 17000:
            return 'cdmx'
        elif 50000 <= cp_int <= 58000:
            return 'edomex'
        else:
            return 'foraneo'
    
    # Crear una copia del DataFrame para no modificar el original
    df_copy = df.copy()
    
    # Inicializar lista para almacenar las nuevas etiquetas
    nuevas_etiquetas = []
    
    for idx, row in df_copy.iterrows():
        cp = row['CP']
        cp_index = row['CP_INDEX']
        entidad_actual = row['ENTIDAD']
        
        # Verificar si CP_INDEX está vacío/nulo
        cp_index_vacio = (
            pd.isna(cp_index) or 
            cp_index == '' or 
            str(cp_index).strip() == '' or
            str(cp_index).strip() == 'nan'
        )
        
        if cp_index_vacio:
            # Caso 1: CP_INDEX vacío → usar solo CP
            etiqueta = obtener_etiqueta_cp(cp)
            nuevas_etiquetas.append(etiqueta if etiqueta else entidad_actual)
        
        else:
            # Ambos campos tienen valores
            cp_limpio = str(cp).strip()
            cp_index_limpio = str(cp_index).strip()
            
            if cp_limpio == cp_index_limpio:
                # Caso 2: Ambos valores son iguales
                etiqueta = obtener_etiqueta_cp(cp)
                nuevas_etiquetas.append(etiqueta if etiqueta else entidad_actual)
            
            else:
                # Caso 3: Valores diferentes → comparar etiquetas
                etiqueta_cp = obtener_etiqueta_cp(cp)
                etiqueta_cp_index = obtener_etiqueta_cp(cp_index)
                
                # Si ambas etiquetas son válidas y son iguales
                if (etiqueta_cp and etiqueta_cp_index and 
                    etiqueta_cp == etiqueta_cp_index):
                    nuevas_etiquetas.append(etiqueta_cp)
                else:
                    # Etiquetas diferentes o alguna inválida → mantener actual
                    nuevas_etiquetas.append(entidad_actual)
    
    # Asignar las nuevas etiquetas
    df_copy['ENTIDAD'] = nuevas_etiquetas
    
    return df_copy



In [28]:
def aplicar_asignacion_entidad_a_datasets():
    """
    Aplica la asignación de entidades a todos los DataFrames en original_datasets
    con estadísticas detalladas
    """
    print("=== Aplicando asignación de ENTIDAD por código postal ===")
    print("-" * 70)
    
    resultados = {}
    resumen_final = {'cdmx': 0, 'edomex': 0, 'foraneo': 0, 'otros': 0}
    
    datasets_a_procesar = {
        name: df for name, df in original_datasets.items() 
        if not name.endswith('_2023')  # Excluir datasets que terminen en _2023
    }
    for dataset_name, df in datasets_a_procesar.items():
        print(f"📊 Procesando: {dataset_name}")
        
        # Contar valores antes
        conteo_antes = df['ENTIDAD'].value_counts()
        print(f"  📋 Antes: {dict(conteo_antes)}")
        
        # Aplicar la función
        df_actualizado = asignar_entidad_por_codigo_postal(df)
        
        # Contar valores después
        conteo_despues = df_actualizado['ENTIDAD'].value_counts()
        print(f"  ✅ Después: {dict(conteo_despues)}")
        
        # Estadísticas específicas de cdmx, edomex, foraneo
        cdmx_count = conteo_despues.get('cdmx', 0)
        edomex_count = conteo_despues.get('edomex', 0)
        foraneo_count = conteo_despues.get('foraneo', 0)
        otros_count = len(df_actualizado) - (cdmx_count + edomex_count + foraneo_count)
        
        print(f"  🏙️  CDMX: {cdmx_count:,} ({cdmx_count/len(df_actualizado)*100:.1f}%)")
        print(f"  🏘️  Estado de México: {edomex_count:,} ({edomex_count/len(df_actualizado)*100:.1f}%)")
        print(f"  🌍 Foráneo: {foraneo_count:,} ({foraneo_count/len(df_actualizado)*100:.1f}%)")
        if otros_count > 0:
            print(f"  ❓ Otros: {otros_count:,} ({otros_count/len(df_actualizado)*100:.1f}%)")
        
        # Calcular cambios
        cambios = len(df) - (df['ENTIDAD'] == df_actualizado['ENTIDAD']).sum()
        print(f"  🔄 Registros modificados: {cambios:,}")
        
        # Acumular para resumen final
        resumen_final['cdmx'] += cdmx_count
        resumen_final['edomex'] += edomex_count
        resumen_final['foraneo'] += foraneo_count
        resumen_final['otros'] += otros_count
        
        print("-" * 50)
        
        # Actualizar el dataset
        original_datasets[dataset_name] = df_actualizado
        
        resultados[dataset_name] = {
            'antes': dict(conteo_antes),
            'despues': dict(conteo_despues),
            'cambios': cambios,
            'estadisticas_entidad': {
                'cdmx': cdmx_count,
                'edomex': edomex_count,
                'foraneo': foraneo_count,
                'otros': otros_count
            },
            'porcentajes': {
                'cdmx': round(cdmx_count/len(df_actualizado)*100, 2),
                'edomex': round(edomex_count/len(df_actualizado)*100, 2),
                'foraneo': round(foraneo_count/len(df_actualizado)*100, 2),
                'otros': round(otros_count/len(df_actualizado)*100, 2) if otros_count > 0 else 0
            }
        }
    
    # Mostrar resumen final consolidado
    total_registros = sum(resumen_final.values())
    print("=" * 70)
    print("📈 RESUMEN CONSOLIDADO DE TODOS LOS DATASETS")
    print("=" * 70)
    print(f"🏙️  CDMX: {resumen_final['cdmx']:,} ({resumen_final['cdmx']/total_registros*100:.1f}%)")
    print(f"🏘️  Estado de México: {resumen_final['edomex']:,} ({resumen_final['edomex']/total_registros*100:.1f}%)")
    print(f"🌍 Foráneo: {resumen_final['foraneo']:,} ({resumen_final['foraneo']/total_registros*100:.1f}%)")
    if resumen_final['otros'] > 0:
        print(f"❓ Otros: {resumen_final['otros']:,} ({resumen_final['otros']/total_registros*100:.1f}%)")
    print(f"📊 Total de registros: {total_registros:,}")
    print("=" * 70)
    
    print("✅ Proceso completado")
    
    # Agregar resumen final a los resultados
    resultados['resumen_consolidado'] = {
        'totales': resumen_final,
        'porcentajes_globales': {
            'cdmx': round(resumen_final['cdmx']/total_registros*100, 2),
            'edomex': round(resumen_final['edomex']/total_registros*100, 2),
            'foraneo': round(resumen_final['foraneo']/total_registros*100, 2),
            'otros': round(resumen_final['otros']/total_registros*100, 2) if resumen_final['otros'] > 0 else 0
        },
        'total_registros': total_registros
    }
    
    return resultados

In [29]:
# Aplicar la asignación con estadísticas detalladas
resultados = aplicar_asignacion_entidad_a_datasets()

# Acceder a estadísticas específicas de un dataset
print("\n🔍 Ejemplo de acceso a estadísticas específicas:")
for dataset_name in ['df_coordenadas_sustentantes_2020']:
    stats = resultados[dataset_name]['estadisticas_entidad']
    print(f"{dataset_name}:")
    print(f"  - CDMX: {stats['cdmx']:,}")
    print(f"  - Estado de México: {stats['edomex']:,}")
    print(f"  - Foráneo: {stats['foraneo']:,}")

=== Aplicando asignación de ENTIDAD por código postal ===
----------------------------------------------------------------------
📊 Procesando: df_coordenadas_sustentantes_2020
  📋 Antes: {'cdmx': np.int64(13235), 'edomex': np.int64(12474)}
  ✅ Después: {'edomex': np.int64(15958), 'cdmx': np.int64(9146), 'foraneo': np.int64(605)}
  🏙️  CDMX: 9,146 (35.6%)
  🏘️  Estado de México: 15,958 (62.1%)
  🌍 Foráneo: 605 (2.4%)
  🔄 Registros modificados: 8,681
--------------------------------------------------
📊 Procesando: df_coordenadas_sustentantes_2021
  📋 Antes: {'cdmx': np.int64(12495), 'edomex': np.int64(12468)}
  ✅ Después: {'edomex': np.int64(15448), 'cdmx': np.int64(8961), 'foraneo': np.int64(554)}
  🏙️  CDMX: 8,961 (35.9%)
  🏘️  Estado de México: 15,448 (61.9%)
  🌍 Foráneo: 554 (2.2%)
  🔄 Registros modificados: 8,205
--------------------------------------------------
📊 Procesando: df_coordenadas_sustentantes_2022
  📋 Antes: {'edomex': np.int64(13796)}
  ✅ Después: {'edomex': np.int64(11

In [30]:
def corregir_registros_extranjeros():
    """
    Muy pocos pero tambien hay extranjeros que aparecen en los datos
    Corrige registros donde COLONIA es 'extranjero' para que ENTIDAD y MUNICIPIO también sean 'extranjero'
    """
    print("=== Corrigiendo registros de extranjeros ===")
    print("-" * 50)
    
    total_cambios = 0
    
    for dataset_name, df in original_datasets.items():
        # Buscar registros con COLONIA = 'extranjero' (case insensitive)
        mask_extranjero = df['COLONIA'].str.lower().str.strip() == 'extranjero'
        
        registros_encontrados = mask_extranjero.sum()
        
        if registros_encontrados > 0:
            print(f"📊 Dataset: {dataset_name}")
            print(f"  🔍 Registros encontrados: {registros_encontrados}")
            
            # Mostrar algunos ejemplos antes del cambio
            ejemplos_antes = df.loc[mask_extranjero, ['COLONIA', 'MUNICIPIO', 'ENTIDAD']].head(3)
            print(f"  📋 Ejemplos antes:")
            for idx, row in ejemplos_antes.iterrows():
                print(f"    - COLONIA: {row['COLONIA']}, MUNICIPIO: {row['MUNICIPIO']}, ENTIDAD: {row['ENTIDAD']}")
            
            # Aplicar correcciones
            df.loc[mask_extranjero, 'ENTIDAD'] = 'extranjero'
            df.loc[mask_extranjero, 'MUNICIPIO'] = 'extranjero'
            
            print(f"  ✅ Registros corregidos: {registros_encontrados}")
            total_cambios += registros_encontrados
            print("-" * 30)
    
    print(f"🎯 Total de registros corregidos: {total_cambios}")
    print("✅ Proceso completado")
    
    return total_cambios



In [31]:
cambios_realizados = corregir_registros_extranjeros()

=== Corrigiendo registros de extranjeros ===
--------------------------------------------------
📊 Dataset: df_coordenadas_sustentantes_2020
  🔍 Registros encontrados: 2
  📋 Ejemplos antes:
    - COLONIA: extranjero, MUNICIPIO: extranjero, ENTIDAD: cdmx
    - COLONIA: extranjero, MUNICIPIO: extranjero, ENTIDAD: cdmx
  ✅ Registros corregidos: 2
------------------------------
📊 Dataset: df_coordenadas_sustentantes_2021
  🔍 Registros encontrados: 1
  📋 Ejemplos antes:
    - COLONIA: extranjero, MUNICIPIO: extranjero, ENTIDAD: cdmx
  ✅ Registros corregidos: 1
------------------------------
🎯 Total de registros corregidos: 3
✅ Proceso completado


---

Aplicar teorica de conjuntos para corregir inconsistencias en los datos, y de igual forma, para disminuir la complejidad algoritmica de la solución.

---

Primera forma de proceder es separar los valores unicos de los municipios y entidades de los dataframes originales, para luego hacer una union de los valores unicos y asi obtener un catalogo de valores unicos que nos permita corregir los errores en los datos.

In [32]:
arr_municipios_gdf = marginacion_gdf['MUNICIPIO'].unique().tolist()
arr_municipios_catalogo = catalogo_cp['MUNICIPIO'].unique().tolist()
arr_municipios_original_datasets2020 = original_datasets['df_coordenadas_sustentantes_2020']['MUNICIPIO'].unique().tolist()
arr_municipios_original_datasets2021 = original_datasets['df_coordenadas_sustentantes_2021']['MUNICIPIO'].unique().tolist()
arr_municipios_original_datasets2022 = original_datasets['df_coordenadas_sustentantes_2022']['MUNICIPIO'].unique().tolist()
arr_municipios_original_datasets2023 = original_datasets['df_coordenadas_sustentantes_2023']['MUNICIPIO'].unique().tolist()

In [33]:
comunes2020 = np.intersect1d(arr_municipios_gdf, arr_municipios_original_datasets2020)
comunes2021 = np.intersect1d(arr_municipios_gdf, arr_municipios_original_datasets2021)
comunes2022 = np.intersect1d(arr_municipios_gdf, arr_municipios_original_datasets2022)
comunes2023 = np.intersect1d(arr_municipios_gdf, arr_municipios_original_datasets2023)

elements_exclusivos2020 = np.setdiff1d(arr_municipios_original_datasets2020, comunes2020)
elements_exclusivos2021 = np.setdiff1d(arr_municipios_original_datasets2021, comunes2021)
elements_exclusivos2022 = np.setdiff1d(arr_municipios_original_datasets2022, comunes2022)
elements_exclusivos2023 = np.setdiff1d(arr_municipios_original_datasets2023, comunes2023)

comunes2020_prima = np.intersect1d(arr_municipios_catalogo, arr_municipios_original_datasets2020)
comunes2021_prima = np.intersect1d(arr_municipios_catalogo, arr_municipios_original_datasets2021)
comunes2022_prima = np.intersect1d(arr_municipios_catalogo, arr_municipios_original_datasets2022)
comunes2023_prima = np.intersect1d(arr_municipios_catalogo, arr_municipios_original_datasets2023)

elements_exclusivos2020_prima = np.setdiff1d(arr_municipios_original_datasets2020, comunes2020_prima)
elements_exclusivos2021_prima = np.setdiff1d(arr_municipios_original_datasets2021, comunes2021_prima)
elements_exclusivos2022_prima = np.setdiff1d(arr_municipios_original_datasets2022, comunes2022_prima)
elements_exclusivos2023_prima = np.setdiff1d(arr_municipios_original_datasets2023, comunes2023_prima)

print("Cantidad de municipios diferentes entre los datos de referencia y los datasets originales:")
len(elements_exclusivos2020_prima), len(elements_exclusivos2021_prima), len(elements_exclusivos2022_prima), len(elements_exclusivos2023_prima)

Cantidad de municipios diferentes entre los datos de referencia y los datasets originales:


(279, 232, 127, 2)

In [34]:
elements_exclusivos2020_prima, elements_exclusivos2021_prima, elements_exclusivos2022_prima, elements_exclusivos2023_prima

(array(['acajete', 'acambaro', 'acambay', 'acapulco de juarez', 'acatlan',
        'acaxochitlan', 'acayucan', 'actopan', 'aguascalientes',
        'ahuazotepec', 'allende', 'altotonga', 'amealco de bonfil',
        'amixtlan', 'amozoc', 'angel r cabada', 'apaseo el grande',
        'apizaco', 'apodaca', 'arcelia', 'arroyo seco', 'atitalaquia',
        'atlapexco', 'atlatlahucan', 'atlixco', 'atotonilco de tula',
        'atzacan', 'atzitzintla', 'bahia de banderas', 'cadada morelos',
        'calnali', 'calpulalpan', 'cardenas', 'carmen', 'celaya', 'centro',
        'chalchicomula de sesma', 'chapantongo', 'chapulhuacan', 'charcas',
        'chiapa de corzo', 'chiautzingo', 'chicoloapan de juarez',
        'chicontepec', 'chignahuapan', 'chilapa de alvarez', 'chilchotla',
        'chilpancingo de los bravo', 'cintalapa', 'coatepec',
        'coatzacoalcos', 'coatzintla', 'colima', 'comalcalco',
        'comitan de dominguez', 'comonfort', 'concepcion papalo', 'conkal',
        'correg

In [35]:
cammon = set(elements_exclusivos2020_prima) | set(elements_exclusivos2021_prima) | set(elements_exclusivos2022_prima) | set(elements_exclusivos2023_prima)
cammon

{np.str_('acajete'),
 np.str_('acambaro'),
 np.str_('acambay'),
 np.str_('acapulco de juarez'),
 np.str_('acatepec'),
 np.str_('acatlan'),
 np.str_('acaxochitlan'),
 np.str_('acayucan'),
 np.str_('actopan'),
 np.str_('agua dulce'),
 np.str_('aguascalientes'),
 np.str_('ahuazotepec'),
 np.str_('alfajayucan'),
 np.str_('allende'),
 np.str_('altotonga'),
 np.str_('amaxac de guerrero'),
 np.str_('amealco de bonfil'),
 np.str_('amixtlan'),
 np.str_('amozoc'),
 np.str_('angangueo'),
 np.str_('angel r cabada'),
 np.str_('apan'),
 np.str_('apaseo el alto'),
 np.str_('apaseo el grande'),
 np.str_('apetatitlan de a carbajal'),
 np.str_('apizaco'),
 np.str_('apodaca'),
 np.str_('arcelia'),
 np.str_('arroyo seco'),
 np.str_('asuncion ixtaltepec'),
 np.str_('asuncion nochixtlan'),
 np.str_('atitalaquia'),
 np.str_('atlapexco'),
 np.str_('atlatlahucan'),
 np.str_('atlixco'),
 np.str_('atotonilco de tula'),
 np.str_('atzacan'),
 np.str_('atzitzintla'),
 np.str_('ayahualulco'),
 np.str_('ayala'),
 np.

In [36]:
def cambiar_valores_columna(df, columna, diccionario_reemplazos):
    """
    Despues de revisar los municipios que no aparecen, se corrigen de forma puntual los nombres de los municipios
    y sus entidades correspondientes en los datasets originales.
    
    Args:
        df: cada dataframe de los datasets originales
        columna: nombre de la columna a modificar
        diccionario_reemplazos: diccionario con los reemplazos a realizar
    """
    df_copy = df.copy()
    df_copy[columna] = df_copy[columna].replace(diccionario_reemplazos)
    return df_copy

# Ejemplo de uso
reemplazos_municipios = {
    "chicoloapan de juarez": "chicoloapan",
    'acambay': 'acambay de ruiz castaneda',
    'morelia': 'la paz',
    'zaragoza': 'atizapan de zaragoza',
    'acatepec': 'ecatepec de morelos', #habian quienes lo escribian como acatepec, y ecatepec de morelos es de los municipios con mayor participación en el examen
    'ecatepec': 'ecatepec de morelos',
    'tlalnepantla': 'tlalnepantla de baz',
    'susupuato': 'tecamac',
    'chihuahua': 'chimalhuacan',
    'matlapa':'naucalpan de juarez',
    'los cabos': 'naucalpan de juarez',
    'nicolas bravo': 'valle de chalco solidaridad'
}

for dataset_name, df in original_datasets.items():
    original_datasets[dataset_name] = cambiar_valores_columna(df, "MUNICIPIO", reemplazos_municipios)


In [37]:
# Cambiar el registro de forma especifica despues de ser revisados.
original_datasets["df_coordenadas_sustentantes_2023"].loc[17054, 'ENTIDAD'] = 'foraneo'
original_datasets["df_coordenadas_sustentantes_2021"].loc[18953, 'MUNICIPIO'] = 'chalco'
original_datasets["df_coordenadas_sustentantes_2021"].loc[18953, 'COLONIA'] = 'culturas de mexico'
original_datasets["df_coordenadas_sustentantes_2020"].loc[18998, 'COLONIA'] = 'canaditas'

In [38]:
def actualizar_entidades_todos_datasets():
    """
    Actualiza el campo 'ENTIDAD' para todos los datasets basado en los municipios
    """
    cdmx = [
        "venustiano carranza", "iztapalapa", "azcapotzalco", "miguel hidalgo",
        "tlalpan", "alvaro obregon", "coyoacan", "gustavo a madero",
        "cuauhtemoc", "xochimilco"
    ]

    edomex = [
        "naucalpan de juarez", "atizapan de zaragoza", "texcoco", "hueypoxtla",
        "chiautla", "tlalnepantla de baz", "tultitlan", "coacalco de berriozabal",
        "cuautitlan", "cuautitlan izcalli", "melchor ocampo", "tepotzotlan",
        "chalco", "ixtapaluca", "ecatepec de morelos", "nezahualcoyotl",'huixquilucan',
        'chimalhuacan','nicolas romero','valle de chalco solidaridad'
        
    ]
    
    total_cambios = 0
    
    for dataset_name, df in original_datasets.items():
        print(f"\n📊 Procesando: {dataset_name}")
        
        # Crear máscaras
        mask_cdmx = df['MUNICIPIO'].str.lower().isin(cdmx)
        mask_edomex = df['MUNICIPIO'].str.lower().isin(edomex)
        
        # Contar cambios antes de aplicar
        cambios_cdmx = mask_cdmx.sum()
        cambios_edomex = mask_edomex.sum()
        
        # Aplicar cambios
        df.loc[mask_cdmx, 'ENTIDAD'] = 'cdmx'
        df.loc[mask_edomex, 'ENTIDAD'] = 'edomex'
        
        print(f"  🏙️  CDMX: {cambios_cdmx} registros")
        print(f"  🏘️  Estado de México: {cambios_edomex} registros")
        
        total_cambios += cambios_cdmx + cambios_edomex
    
    print(f"\n✅ Total de registros actualizados: {total_cambios}")
    return total_cambios



In [39]:
cambios_totales = actualizar_entidades_todos_datasets()


📊 Procesando: df_coordenadas_sustentantes_2020
  🏙️  CDMX: 7631 registros
  🏘️  Estado de México: 12390 registros

📊 Procesando: df_coordenadas_sustentantes_2021
  🏙️  CDMX: 7528 registros
  🏘️  Estado de México: 11917 registros

📊 Procesando: df_coordenadas_sustentantes_2022
  🏙️  CDMX: 2050 registros
  🏘️  Estado de México: 8398 registros

📊 Procesando: df_coordenadas_sustentantes_2023
  🏙️  CDMX: 5796 registros
  🏘️  Estado de México: 8525 registros

✅ Total de registros actualizados: 64235


In [40]:
# Despues de la corrección de los municipios, ahora podemos obtener los municipios problemáticos que son muy probablemente foraneos, es decir, aquellos que no están en el índice de marginación ni en el catálogo de códigos postales.
# Crear conjuntos base (más eficiente que arrays para operaciones de conjuntos)
municipios_marginacion = set(marginacion_gdf['MUNICIPIO'].unique())
municipios_catalogo = set(catalogo_cp['MUNICIPIO'].unique())

# Crear la unión de ambos conjuntos de referencia
municipios_referencia = municipios_marginacion.union(municipios_catalogo)

# Función para obtener municipios problemáticos por año
def obtener_municipios_problematicos(year):
    """
    Retorna municipios del dataset del año especificado que NO están
    ni en marginacion_gdf ni en catalogo_cp
    """
    municipios_dataset = set(original_datasets[f'df_coordenadas_sustentantes_{year}']['MUNICIPIO'].unique())
    return municipios_dataset - municipios_referencia

# Obtener municipios problemáticos para cada año
municipios_problematicos = {
    year: obtener_municipios_problematicos(year) 
    for year in ['2020', '2021', '2022', '2023']
}

# Mostrar resultados
for year, problematicos in municipios_problematicos.items():
    print(f"Año {year}: {len(problematicos)} municipios problemáticos")
    if problematicos:
        print(f"  Ejemplos: {list(problematicos)[:]}")  
    print()

# Si quieres obtener la longitud de cada conjunto como en tu código original
longitudes = tuple(len(municipios_problematicos[year]) for year in ['2020', '2021', '2022', '2023'])
print(f"Longitudes: {longitudes}")

Año 2020: 272 municipios problemáticos
  Ejemplos: ['mecatlan', 'yahualica', 'tezontepec de aldama', 'chilapa de alvarez', 'mexicali', 'santa maria del tule', 'tehuacan', 'leon', 'san andres solaga', 'tehuipango', 'panuco', 'emiliano zapata', 'cutzamala de pinzon', 'uruapan', 'xicotepec', 'pachuca de soto', 'san antonio de la cal', 'nativitas', 'tarandacuao', 'oaxaca de juarez', 'puerto vallarta', 'medellin', 'temixco', 'juan galindo', 'izucar de matamoros', 'maravatio', 'charcas', 'cuautlancingo', 'xico', 'atzacan', 'san pedro mixtepec-juquila', 'coatzacoalcos', 'zacapoaxtla', 'acapulco de juarez', 'yauhquemecan', 'santiago jocotepec', 'tuxtla gutierrez', 'santo domingo tehuantepec', 'ixhuatlancillo', 'huayacocotla', 'colima', 'apodaca', 'santiago maravatio', 'platon sanchez', 'tocumbo', 'zacapu', 'san joaquin', 'san salvador el seco', 'othon p blanco', 'san francisco del rincon', 'san nicolas buenos aires', 'chignahuapan', 'comonfort', 'chapulhuacan', 'tetela del volcan', 'tlajomulco

### 1. Primera metodologia:
Es la optención del indice de marginacion a partir de la distancia minima los centroides de los poligonos y de los centros de las coordenadas  reportadas en los datasests de las cooordenadas sustentantes, el problema es saber que dichas coordenadas hayan sido obtenidas de manera correcta.

In [ ]:
Aqui me falta bien revisar con robustes para estar seguro de estar trabajando con las mismas unidades, el lunes lo tenia ambos en grados, pero considere que seria mejor trabajar con metros, por lo que ahora estoy usando el CRS 32614 que es UTM zona 14N, y los datos de marginación estan en EPSG:4326, por lo que hay que hacer la conversión de CRS para poder calcular las distancias entre puntos.

In [63]:
def detectar_sistema_coordenadas_automatico(original_datasets):
    """
    Detecta automáticamente el sistema de coordenadas más probable
    """
    print("=== Detección Automática de Sistema de Coordenadas ===")
    print("-" * 60)
    
    recomendaciones = {}
    
    for dataset_name, df in original_datasets.items():
        # Tomar muestra para análisis
        sample_df = df.sample(min(1000, len(df)), random_state=42)
        
        lat_mean = sample_df['lat_centro'].mean()
        lng_mean = sample_df['lng_centro'].mean()
        lat_std = sample_df['lat_centro'].std()
        lng_std = sample_df['lng_centro'].std()
        
        # Criterios de decisión
        criterios = {
            'WGS84_Valle_Mexico': (
                18.0 <= lat_mean <= 21.0 and 
                -101.0 <= lng_mean <= -98.0 and
                lat_std < 2.0 and lng_std < 2.0
            ),
            'UTM_Zone_14N': (
                400000 <= lat_mean <= 600000 and 
                2000000 <= lng_mean <= 2400000
            ),
            'Coordenadas_Invalidas': (
                abs(lat_mean) > 90 or abs(lng_mean) > 180
            )
        }
        
        sistema_detectado = None
        for sistema, cumple in criterios.items():
            if cumple:
                sistema_detectado = sistema
                break
        
        if sistema_detectado is None:
            sistema_detectado = 'Sistema_Desconocido'
        
        recomendaciones[dataset_name] = {
            'sistema_detectado': sistema_detectado,
            'lat_mean': lat_mean,
            'lng_mean': lng_mean,
            'usar_transformacion': sistema_detectado == 'WGS84_Valle_Mexico'
        }
        
        print(f"📊 {dataset_name}:")
        print(f"   Sistema detectado: {sistema_detectado}")
        print(f"   Centro promedio: ({lat_mean:.6f}, {lng_mean:.6f})")
        print(f"   Usar transformación CRS: {'Sí' if recomendaciones[dataset_name]['usar_transformacion'] else 'No'}")
        print("-" * 30)
    
    return recomendaciones

# Ejecutar detección automática
recomendaciones = detectar_sistema_coordenadas_automatico(original_datasets)

=== Detección Automática de Sistema de Coordenadas ===
------------------------------------------------------------
📊 df_coordenadas_sustentantes_2020:
   Sistema detectado: WGS84_Valle_Mexico
   Centro promedio: (19.421484, -99.114396)
   Usar transformación CRS: Sí
------------------------------
📊 df_coordenadas_sustentantes_2021:
   Sistema detectado: WGS84_Valle_Mexico
   Centro promedio: (19.475735, -99.139639)
   Usar transformación CRS: Sí
------------------------------
📊 df_coordenadas_sustentantes_2022:
   Sistema detectado: WGS84_Valle_Mexico
   Centro promedio: (19.500475, -99.122029)
   Usar transformación CRS: Sí
------------------------------
📊 df_coordenadas_sustentantes_2023:
   Sistema detectado: WGS84_Valle_Mexico
   Centro promedio: (19.464698, -99.102815)
   Usar transformación CRS: Sí
------------------------------


In [47]:
# Aplicar el método a cada fila usando apply
marginacion_gdf["punto_centro_dentro"] = marginacion_gdf["geometry"].apply(lambda geom: geom.representative_point())
## Representative_point() devuelve un punto representativo dentro de la geometría, que es útil para evitar problemas de geometrías complejas o multipolígonos, ya que es un centroide representativo dentro del multipoligo pero podemos usar  geom.centroid, para obtener el centroide real.

# Extraer coordenadas del punto representativo
marginacion_gdf["latitud_centro"] = marginacion_gdf["punto_centro_dentro"].apply(lambda punto: punto.y)
marginacion_gdf["longitud_centro"] = marginacion_gdf["punto_centro_dentro"].apply(lambda punto: punto.x)


In [49]:

def calcular_distancia_minima_optimizada(original_datasets, marginacion_gdf, usar_utm=True):
    """
    Versión optimizada que calcula distancias mínimas usando BallTree
    y maneja unidades correctamente (metros/km en lugar de grados)
    """
    
    if usar_utm:
        # Opción A: Usar proyección UTM para el Valle de México
        print("🌍 Transformando coordenadas a UTM Zone 14N...")
        marginacion_utm = marginacion_gdf.to_crs(epsg=32614)
        
        # Extraer coordenadas en metros
        coords_marginacion = marginacion_utm[['latitud_centro', 'longitud_centro']].values
        gm_2020_values = marginacion_utm['GM_2020'].values
        
        print("=== Calculando distancias mínimas (UTM - metros) ===")
        
    else:
        # Opción B: Usar BallTree con métrica haversine (coordenadas en radianes)
        print("🌍 Usando coordenadas geográficas con métrica Haversine...")
        coords_marginacion_rad = np.radians(
            marginacion_gdf[['latitud_centro', 'longitud_centro']].values
        )
        gm_2020_values = marginacion_gdf['GM_2020'].values
        
        print("=== Calculando distancias mínimas (Haversine - km) ===")
    
    print(f"📍 Puntos de referencia (marginación): {len(coords_marginacion if usar_utm else coords_marginacion_rad):,}")
    print("-" * 50)
    
    # Crear BallTree
    if usar_utm:
        # Para UTM usamos métrica euclidiana (coordenadas ya están en metros)
        tree = BallTree(coords_marginacion, metric='euclidean')
    else:
        # Para lat/lon usamos haversine (coordenadas en radianes)
        tree = BallTree(coords_marginacion_rad, metric='haversine')
    
    # Procesar cada dataset
    for dataset_name, df in original_datasets.items():
        print(f"🔄 Procesando: {dataset_name}")
        print(f"  📊 Registros: {len(df):,}")
        
        if usar_utm:
            # Crear GeoDataFrame temporal para transformar coordenadas
            import geopandas as gpd
            from shapely.geometry import Point
            
            # Crear puntos en WGS84
            geometry = [Point(lng, lat) for lng, lat in zip(df['lng_centro'], df['lat_centro'])]
            gdf_temp = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')
            
            # Transformar a UTM
            gdf_utm = gdf_temp.to_crs(epsg=32614)
            
            # Extraer coordenadas UTM
            coords_dataset = np.array([[geom.x, geom.y] for geom in gdf_utm.geometry])
            
        else:
            # Convertir a radianes para haversine
            coords_dataset_rad = np.radians(
                df[['lat_centro', 'lng_centro']].values
            )
            coords_dataset = coords_dataset_rad
        
        # Buscar vecinos más cercanos usando BallTree
        print("  🔍 Buscando vecinos más cercanos...")
        distancias, indices = tree.query(coords_dataset, k=1)
        
        # Aplanar arrays (query devuelve arrays 2D)
        distancias_minimas = distancias.flatten()
        indices_minimos = indices.flatten()
        
        if not usar_utm:
            # Convertir de radianes a kilómetros (radio de la Tierra ≈ 6371 km)
            distancias_minimas = distancias_minimas * 6371
        else:
            # Convertir de metros a kilómetros
            distancias_minimas = distancias_minimas / 1000
        
        # Asignar valores correspondientes
        df['IM_DISTANCE'] = gm_2020_values[indices_minimos]
        df['Distance'] = distancias_minimas
        
        # Estadísticas mejoradas
        dist_promedio = np.mean(distancias_minimas)
        dist_mediana = np.median(distancias_minimas)
        dist_max = np.max(distancias_minimas)
        dist_min = np.min(distancias_minimas)
        dist_std = np.std(distancias_minimas)
        
        unidad = "km"
        print(f"  📏 Distancia promedio: {dist_promedio:.3f} {unidad}")
        print(f"  📐 Distancia mediana: {dist_mediana:.3f} {unidad}")
        print(f"  📈 Distancia máxima: {dist_max:.3f} {unidad}")
        print(f"  📉 Distancia mínima: {dist_min:.3f} {unidad}")
        print(f"  📊 Desviación estándar: {dist_std:.3f} {unidad}")
        
        # Análisis de distribución de distancias
        rangos_dist = {
            '< 1 km': np.sum(distancias_minimas < 1),
            '1-5 km': np.sum((distancias_minimas >= 1) & (distancias_minimas < 5)),
            '5-10 km': np.sum((distancias_minimas >= 5) & (distancias_minimas < 10)),
            '> 10 km': np.sum(distancias_minimas >= 10)
        }
        
        print(f"  📊 Distribución de distancias:")
        for rango, cantidad in rangos_dist.items():
            if cantidad > 0:
                porcentaje = (cantidad / len(distancias_minimas)) * 100
                print(f"    - {rango}: {cantidad:,} ({porcentaje:.1f}%)")
        
        # Mostrar distribución de GM_2020 asignados
        gm_counts = df['IM_DISTANCE'].value_counts().head(5)
        print(f"  🎯 Top 5 GM_2020 asignados:")
        for gm_value, count in gm_counts.items():
            print(f"    - {gm_value}: {count:,} registros")
        
        print("-" * 30)
    
    print("✅ Proceso completado")
    return original_datasets


In [51]:
marginacion_gdf

,GM_2020,COLONIA,CP,MUNICIPIO,ENTIDAD,geometry,latitud_centro,longitud_centro,punto_centro_dentro
0,Bajo,aguilera,02900,azcapotzalco,cdmx,"MULTIPOLYGON (((-99.15443 19.47346, -99.15616 ...",19.472887,-99.155905,POINT (-99.15591 19.47289)
1,Bajo,aldana,02910,azcapotzalco,cdmx,"MULTIPOLYGON (((-99.1461 19.47323, -99.1473 19...",19.470644,-99.149993,POINT (-99.14999 19.47064)
2,Muy bajo,ampliacion del gas,02970,azcapotzalco,cdmx,"MULTIPOLYGON (((-99.15522 19.46844, -99.15634 ...",19.466378,-99.157475,POINT (-99.15747 19.46638)
3,Bajo,ampliacion petrolera,02470,azcapotzalco,cdmx,"MULTIPOLYGON (((-99.1965 19.4852, -99.19667 19...",19.483468,-99.195337,POINT (-99.19534 19.48347)
4,Medio,ampliacion san pedro xalpa,02719,azcapotzalco,cdmx,"MULTIPOLYGON (((-99.21601 19.48404, -99.21667 ...",19.478625,-99.216786,POINT (-99.21679 19.47862)
...,...,...,...,...,...,...,...,...,...
5955,Bajo,jajalpa,54980,tultepec,edomex,"MULTIPOLYGON (((-99.1241 19.6583, -99.12502 19...",19.657250,-99.125990,POINT (-99.12599 19.65725)
5956,Medio,la providencia,54980,tultepec,edomex,"MULTIPOLYGON (((-99.13675 19.64888, -99.13642 ...",19.649216,-99.137092,POINT (-99.13709 19.64922)
5957,Bajo,santiago teyahualco,54980,tultepec,edomex,"MULTIPOLYGON (((-99.11802 19.66253, -99.11865 ...",19.658982,-99.121330,POINT (-99.12133 19.65898)
5958,Medio,ex hda la mariscala,54949,tultitlan,edomex,"MULTIPOLYGON (((-99.13577 19.62349, -99.13583 ...",19.624258,-99.136236,POINT (-99.13624 19.62426)


In [50]:
# Ejecutar la función
original_datasets = calcular_distancia_minima_optimizada(original_datasets, marginacion_gdf)

🌍 Transformando coordenadas a UTM Zone 14N...
=== Calculando distancias mínimas (UTM - metros) ===
📍 Puntos de referencia (marginación): 5,960
--------------------------------------------------
🔄 Procesando: df_coordenadas_sustentantes_2020
  📊 Registros: 25,709
  🔍 Buscando vecinos más cercanos...


ValueError: Input contains NaN.

In [ ]:
original_datasets["df_coordenadas_sustentantes_2023"]

,COLONIA,CP,MUNICIPIO,ENTIDAD,lat_centro,lng_centro,lat_principal,lng_principal,CP_INDEX,IM_2020,HASH_CP_gpkg,IM_DISTANCE,Distance,HASH_CP_Catalogo,TIPO_ASENTAMIENTO,TIPO_DE_ZONA
0,san andres tetepilco,09440,iztapalapa,cdmx,19.371723,-99.132493,19.373162,-99.130396,09440,Bajo,09850,Medio,0.002334,,,
1,san antonio xahuento,54976,tultepec,edomex,19.690464,-99.112582,19.692443,-99.111031,54976,Alto,54960,Alto,0.002537,,,
2,barrio la asuncion,16040,xochimilco,cdmx,19.271636,-99.101098,19.271636,-99.101098,16040,Medio,16040,Medio,0.002303,,,
3,modelo,57530,nezahualcoyotl,edomex,19.402045,-99.030601,19.402102,-99.030507,57530,Medio,57530,Medio,0.000062,,,
4,hogares marla,55030,ecatepec de morelos,edomex,19.604641,-99.049833,19.604515,-99.049853,55030,Medio,55040,Bajo,0.000057,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18688,alamos,03410,benito juarez,cdmx,19.390471,-99.142494,19.391054,-99.140772,03410,Muy bajo,03400,Muy bajo,0.001309,,,
18689,cobadonga,56607,chalco,edomex,19.275884,-98.910711,19.277908,-98.912117,56607,Alto,56607,Alto,0.001185,,,
18690,gustavo baz prada,54130,tlalnepantla de baz,edomex,19.561345,-99.175959,19.561611,-99.176754,54120,Medio,54120,Medio,0.000563,,,
18691,fraccarbolada los sauces 2,55635,zumpango,edomex,19.815479,-99.042517,19.815731,-99.042252,55635,Medio,55600,Medio,0.002233,,,


: 

### 2. Segunda metodologia:
Para esta segunda metodologia para optención del indice de marginacion del geodataframe `margiinación_gdf`, se usan enfoques con dos librerias diferentes `fuzzywuzzy` y `recordlinkage`.

Con `fuzzywuzzy` se comparan su diferentes funciones y se usa la de `fuzz.token_sort_ratio` para comparar los nombres de las colonias, y se usa un umbral de 80% para considerar que son iguales, esto se basa en la distancia de levenshtein.

Con `recordlinkage` se usa el metodo de `block` para agrupar los registros por entidad y municipio, y luego se usa el metodo de `compare` con el metodo de `jarowikler`para comparar los nombres de las colonias, este es un metodo de comparacion de cadenas un poco mas robusto que el de `fuzzywuzzy` dado a que:


* Esta basado en la métrica Jaro, que mide cuántos caracteres “coinciden” en dos cadenas y penaliza las transposiciones.

* Jaro–Winkler añade un “bonus” si las cadenas comparten un prefijo común, reforzando la puntuación cuando coinciden los primeros caracteres.

* Es muy útil para nombres o palabras cortas (como “colonias”), donde a menudo hay pequeñas faltas de ortografía, o transposiciones.

Con este metodo se obtiene un promedio del 80% de completes de los datos.

In [64]:


def realizar_record_linkage_colonias(original_datasets, marginacion_gdf):
    """
    Realiza record linkage entre original_datasets y marginacion_gdf basado en:
    - Coincidencia exacta: MUNICIPIO y ENTIDAD
    - Similitud de strings: COLONIA (usando Jarowinkler con score continuo)
    
    Asigna GM_2020 -> IM_2020 y CP -> HASH_CP_gpkg donde hay coincidencias
    """
    
    def normalize_series(s):
        # 1) Limpia acentos y normaliza unicode
        s = clean(s)
        # 2) Minúsculas, elimina todo lo que no sea letra/número/espacio, recorta
        return (
            s
            .str.lower()
            .str.replace(r"[^\w\s]", "", regex=True)
            .str.strip()
        )
    
    print("=== Iniciando Record Linkage para COLONIAS ===")
    print("-" * 60)
    
    # 1) Preparamos marginacion_gdf
    marginacion_clean = marginacion_gdf[[
        'COLONIA', 'MUNICIPIO', 'ENTIDAD', 'GM_2020', 'CP'
    ]].copy()
    marginacion_clean['COLONIA']   = normalize_series(marginacion_clean['COLONIA'])
    marginacion_clean['MUNICIPIO'] = normalize_series(marginacion_clean['MUNICIPIO'])
    marginacion_clean['ENTIDAD']   = normalize_series(marginacion_clean['ENTIDAD'])
    
    resultados_linkage = {}
    
    for dataset_name, df in original_datasets.items():
        print(f"🔄 Procesando: {dataset_name} ({len(df):,} registros)")
        
        # 2) Preparamos cada df
        df_clean = df[['COLONIA', 'MUNICIPIO', 'ENTIDAD']].copy()
        df_clean['COLONIA']   = normalize_series(df_clean['COLONIA'])
        df_clean['MUNICIPIO'] = normalize_series(df_clean['MUNICIPIO'])
        df_clean['ENTIDAD']   = normalize_series(df_clean['ENTIDAD'])
        
        # 3) Indexación por bloque
        indexer = recordlinkage.Index()
        indexer.block(['MUNICIPIO', 'ENTIDAD'])
        candidate_pairs = indexer.index(df_clean, marginacion_clean)
        print(f"  🔍 Pares candidatos: {len(candidate_pairs):,}")
        if candidate_pairs.empty:
            print("  ⚠️  Sin candidatos en este dataset.")
            continue
        
        # 4) Comparador
        comparer = recordlinkage.Compare()
        comparer.exact('MUNICIPIO', 'MUNICIPIO', label='municipio_match')
        comparer.exact('ENTIDAD',   'ENTIDAD',   label='entidad_match')
        comparer.string(
            'COLONIA', 'COLONIA',
            method='jarowinkler',
            threshold=None,             # score continuo [0,1]
            label='colonia_similarity'
        )
        
        print("  🧮 Calculando similitudes...")
        features = comparer.compute(candidate_pairs, df_clean, marginacion_clean)
        
        # 5) Filtrado con umbral
        umbral = 0.8
        mask = (
            (features['municipio_match'] == 1) &
            (features['entidad_match']   == 1) &
            (features['colonia_similarity'] >= umbral)
        )
        matches = candidate_pairs[mask]
        print(f"  ✅ Matches válidos: {len(matches):,}")
        if matches.empty:
            continue
        
        # 6) Asignaciones
        asign = 0
        for idx_orig, idx_marg in matches:
            gm = marginacion_gdf.iloc[idx_marg]['GM_2020']
            cp = marginacion_gdf.iloc[idx_marg]['CP']
            df_loc = original_datasets[dataset_name]
            df_loc.iat[idx_orig, df_loc.columns.get_loc('IM_2020')]      = gm
            df_loc.iat[idx_orig, df_loc.columns.get_loc('HASH_CP_gpkg')] = cp
            asign += 1
        
        cob = asign / len(df) * 100
        print(f"  📋 Asignaciones: {asign:,}  Cobertura: {cob:.2f}%")
        resultados_linkage[dataset_name] = {
            'pares': len(candidate_pairs),
            'matches': len(matches),
            'asignaciones': asign,
            'cobertura_%': cob
        }
        print("-" * 40)
    
    # 7) Resumen final
    total_asig = sum(r['asignaciones'] for r in resultados_linkage.values())
    total_reg  = sum(len(df) for df in original_datasets.values())
    print(f"🎯 Total asignaciones: {total_asig:,} / {total_reg:,}  ({total_asig/total_reg*100:.2f}% cobertura)")
    print("✅ Proceso completado")
    
    return original_datasets, resultados_linkage


In [ ]:


def realizar_fuzzy_linkage_colonias(original_datasets, marginacion_gdf, umbral_similitud=80):
    """
    Realiza fuzzy matching entre original_datasets y marginacion_gdf usando fuzzywuzzy
    - Coincidencia exacta: MUNICIPIO y ENTIDAD
    - Similitud fuzzy: COLONIA (usando token_sort_ratio)
    
    Asigna GM_2020 -> IM_2020 y CP -> HASH_CP_gpkg donde hay coincidencias
    """
    
    print("=== Iniciando Fuzzy Linkage para COLONIAS ===")
    print(f"🎯 Umbral de similitud: {umbral_similitud}%")
    print("-" * 60)
    
    # Preparar marginacion_gdf con índice para búsqueda eficiente
    marginacion_prep = marginacion_gdf[['COLONIA', 'MUNICIPIO', 'ENTIDAD', 'GM_2020', 'CP']].copy()
    
    # Crear diccionario agrupado por (MUNICIPIO, ENTIDAD) para búsqueda más eficiente
    marginacion_dict = {}
    for idx, row in marginacion_prep.iterrows():
        key = (row['MUNICIPIO'].lower().strip(), row['ENTIDAD'].lower().strip())
        if key not in marginacion_dict:
            marginacion_dict[key] = []
        marginacion_dict[key].append({
            'idx': idx,
            'colonia': row['COLONIA'],
            'gm_2020': row['GM_2020'],
            'cp': row['CP']
        })
    
    print(f"📊 Grupos únicos (Municipio-Entidad) en marginación: {len(marginacion_dict):,}")
    
    resultados_linkage = {}
    
    for dataset_name, df in original_datasets.items():
        print(f"\n🔄 Procesando: {dataset_name}")
        print(f"  📊 Registros en dataset: {len(df):,}")
        
        asignaciones_realizadas = 0
        matches_detallados = []
        
        # Procesar cada registro del dataset
        for idx, row in df.iterrows():
            municipio_key = row['MUNICIPIO'].lower().strip()
            entidad_key = row['ENTIDAD'].lower().strip()
            colonia_buscar = row['COLONIA']
            
            # Buscar en el diccionario de marginación
            grupo_key = (municipio_key, entidad_key)
            
            if grupo_key in marginacion_dict:
                mejor_match = None
                mejor_score = 0
                
                # Comparar con todas las colonias del mismo municipio-entidad
                for candidato in marginacion_dict[grupo_key]:
                    # Calcular similitud usando token_sort_ratio
                    score = fuzz.token_sort_ratio(colonia_buscar, candidato['colonia'])
                    
                    if score >= umbral_similitud and score > mejor_score:
                        mejor_score = score
                        mejor_match = candidato
                
                # Si encontramos un match válido, asignar valores
                if mejor_match:
                    # Asignar valores usando iloc para mayor eficiencia
                    df.iloc[idx, df.columns.get_loc('IM_2020')] = mejor_match['gm_2020']
                    df.iloc[idx, df.columns.get_loc('HASH_CP_gpkg')] = mejor_match['cp']
                    
                    asignaciones_realizadas += 1
                    
                    # Guardar detalles del match para estadísticas
                    matches_detallados.append({
                        'colonia_original': colonia_buscar,
                        'colonia_match': mejor_match['colonia'],
                        'municipio': row['MUNICIPIO'],
                        'entidad': row['ENTIDAD'],
                        'score': mejor_score,
                        'gm_2020': mejor_match['gm_2020']
                    })
        
        # Estadísticas del proceso
        cobertura = (asignaciones_realizadas / len(df)) * 100
        print(f"  📋 Asignaciones realizadas: {asignaciones_realizadas:,}")
        print(f"  📊 Cobertura: {cobertura:.2f}%")
        
        # Mostrar ejemplos de matches si los hay
        if matches_detallados:
            print("  🎯 Ejemplos de matches (Top 3 por score):")
            matches_ordenados = sorted(matches_detallados, key=lambda x: x['score'], reverse=True)
            
            for i, match in enumerate(matches_ordenados[:3]):
                print(f"    {i+1}. '{match['colonia_original']}' ↔ '{match['colonia_match']}'")
                print(f"       Municipio: {match['municipio']}, Score: {match['score']}%")
            
            # Estadísticas de scores
            scores = [m['score'] for m in matches_detallados]
            score_promedio = sum(scores) / len(scores)
            score_min = min(scores)
            score_max = max(scores)
            
            print(f"  📈 Score promedio: {score_promedio:.1f}%")
            print(f"  📉 Score mínimo: {score_min}%")
            print(f"  📊 Score máximo: {score_max}%")
            
            # Distribución de scores por rangos
            rangos = {
                '95-100%': len([s for s in scores if s >= 95]),
                '90-94%': len([s for s in scores if 90 <= s < 95]),
                '85-89%': len([s for s in scores if 85 <= s < 90]),
                '80-84%': len([s for s in scores if 80 <= s < 85]),
                f'{umbral_similitud}-79%': len([s for s in scores if umbral_similitud <= s < 80])
            }
            
            print(f"  📊 Distribución de scores:")
            for rango, cantidad in rangos.items():
                if cantidad > 0:
                    porcentaje = (cantidad / len(scores)) * 100
                    print(f"    - {rango}: {cantidad:,} ({porcentaje:.1f}%)")
        
        # Guardar estadísticas
        resultados_linkage[dataset_name] = {
            'asignaciones_realizadas': asignaciones_realizadas,
            'cobertura_porcentaje': cobertura,
            'umbral_usado': umbral_similitud,
            'matches_detallados': len(matches_detallados),
            'score_promedio': score_promedio if matches_detallados else 0,
            'distribución_scores': rangos if matches_detallados else {}
        }
        
        print("-" * 40)
    
    # Resumen final
    print("=" * 60)
    print("📈 RESUMEN FINAL DEL FUZZY LINKAGE")
    print("=" * 60)
    
    total_asignaciones = sum(r['asignaciones_realizadas'] for r in resultados_linkage.values())
    total_registros = sum(len(df) for df in original_datasets.values())
    cobertura_global = (total_asignaciones / total_registros) * 100
    
    print(f"🎯 Total de asignaciones realizadas: {total_asignaciones:,}")
    print(f"📊 Total de registros procesados: {total_registros:,}")
    print(f"📈 Cobertura global: {cobertura_global:.2f}%")
    
    print(f"\n📋 Detalle por dataset:")
    for dataset_name, stats in resultados_linkage.items():
        print(f"  • {dataset_name}: {stats['asignaciones_realizadas']:,} asignaciones ({stats['cobertura_porcentaje']:.2f}% cobertura)")
        if stats['matches_detallados'] > 0:
            print(f"    Score promedio: {stats['score_promedio']:.1f}%")
    
    print("=" * 60)
    print("✅ Proceso de Fuzzy Linkage completado")
    
    return original_datasets, resultados_linkage




: 

In [ ]:
# Función auxiliar para verificar los resultados
def verificar_asignaciones_linkage(original_datasets):
    """
    Verifica y muestra estadísticas de las asignaciones realizadas por record linkage
    """
    print("=== Verificación de Asignaciones por Record Linkage ===")
    print("-" * 60)
    
    for dataset_name, df in original_datasets.items():
        print(f"📊 Dataset: {dataset_name}")
        
        # Contar asignaciones de IM_2020
        im_asignados = df['IM_2020'].notna().sum()
        im_no_vacios = (df['IM_2020'] != '').sum()
        
        # Contar asignaciones de HASH_CP_gpkg  
        hash_asignados = df['HASH_CP_gpkg'].notna().sum()
        hash_no_vacios = (df['HASH_CP_gpkg'] != '').sum()
        
        total_registros = len(df)
        
        print(f"  🎯 IM_2020 asignados: {im_no_vacios:,}/{total_registros:,} ({(im_no_vacios/total_registros)*100:.2f}%)")
        print(f"  🏷️  HASH_CP_gpkg asignados: {hash_no_vacios:,}/{total_registros:,} ({(hash_no_vacios/total_registros)*100:.2f}%)")
        
        # Mostrar algunos ejemplos
        ejemplos_asignados = df[df['IM_2020'] != ''].head(3)
        if not ejemplos_asignados.empty:
            print(f"  📋 Ejemplos de asignaciones:")
            for idx, row in ejemplos_asignados.iterrows():
                print(f"    - COLONIA: {row['COLONIA'][:30]}... | IM_2020: {row['IM_2020']} | CP: {row['HASH_CP_gpkg']}")
        
        print("-" * 40)

: 

In [ ]:
def comparar_fuzzy_methods(colonia1, colonia2):
    """
    Función auxiliar para comparar diferentes métodos de fuzzywuzzy
    Útil para experimentar con diferentes enfoques
    """
    return {
        'ratio': fuzz.ratio(colonia1, colonia2),
        'partial_ratio': fuzz.partial_ratio(colonia1, colonia2),
        'token_sort_ratio': fuzz.token_sort_ratio(colonia1, colonia2),
        'token_set_ratio': fuzz.token_set_ratio(colonia1, colonia2)
    }




: 

In [ ]:
def realizar_fuzzy_linkage_multiple_methods(original_datasets, marginacion_gdf, umbral_similitud=80):
    """
    Versión alternativa que prueba múltiples métodos de fuzzywuzzy y usa el mejor score
    """
    
    print("=== Fuzzy Linkage con Múltiples Métodos ===")
    print(f"🎯 Umbral de similitud: {umbral_similitud}%")
    print("-" * 60)
    
    # Preparar marginacion_gdf
    marginacion_prep = marginacion_gdf[['COLONIA', 'MUNICIPIO', 'ENTIDAD', 'GM_2020', 'CP']].copy()
    
    # Crear diccionario agrupado
    marginacion_dict = {}
    for idx, row in marginacion_prep.iterrows():
        key = (row['MUNICIPIO'].lower().strip(), row['ENTIDAD'].lower().strip())
        if key not in marginacion_dict:
            marginacion_dict[key] = []
        marginacion_dict[key].append({
            'idx': idx,
            'colonia': row['COLONIA'],
            'gm_2020': row['GM_2020'],
            'cp': row['CP']
        })
    
    resultados_linkage = {}
    
    for dataset_name, df in original_datasets.items():
        print(f"\n🔄 Procesando: {dataset_name}")
        print(f"  📊 Registros en dataset: {len(df):,}")
        
        asignaciones_realizadas = 0
        method_usage = {'ratio': 0, 'partial_ratio': 0, 'token_sort_ratio': 0, 'token_set_ratio': 0}
        
        for idx, row in df.iterrows():
            municipio_key = row['MUNICIPIO'].lower().strip()
            entidad_key = row['ENTIDAD'].lower().strip()
            colonia_buscar = row['COLONIA']
            
            grupo_key = (municipio_key, entidad_key)
            
            if grupo_key in marginacion_dict:
                mejor_match = None
                mejor_score = 0
                mejor_method = None
                
                for candidato in marginacion_dict[grupo_key]:
                    # Probar todos los métodos
                    scores = comparar_fuzzy_methods(colonia_buscar, candidato['colonia'])
                    
                    # Encontrar el mejor score entre todos los métodos
                    for method, score in scores.items():
                        if score >= umbral_similitud and score > mejor_score:
                            mejor_score = score
                            mejor_match = candidato
                            mejor_method = method
                
                if mejor_match:
                    df.iloc[idx, df.columns.get_loc('IM_2020')] = mejor_match['gm_2020']
                    df.iloc[idx, df.columns.get_loc('HASH_CP_gpkg')] = mejor_match['cp']
                    asignaciones_realizadas += 1
                    method_usage[mejor_method] += 1
        
        # Estadísticas
        cobertura = (asignaciones_realizadas / len(df)) * 100
        print(f"  📋 Asignaciones realizadas: {asignaciones_realizadas:,}")
        print(f"  📊 Cobertura: {cobertura:.2f}%")
        
        if asignaciones_realizadas > 0:
            print(f"  🔧 Uso de métodos:")
            for method, count in method_usage.items():
                if count > 0:
                    porcentaje = (count / asignaciones_realizadas) * 100
                    print(f"    - {method}: {count:,} ({porcentaje:.1f}%)")
        
        resultados_linkage[dataset_name] = {
            'asignaciones_realizadas': asignaciones_realizadas,
            'cobertura_porcentaje': cobertura,
            'method_usage': method_usage
        }
    
    return original_datasets, resultados_linkage

: 

In [ ]:
# Ejecutar el fuzzy linkage
print("🚀 Iniciando proceso de Fuzzy Linkage...")

# Opción 1: Solo token_sort_ratio (recomendado)
original_datasets, resultados = realizar_fuzzy_linkage_colonias(
    original_datasets, 
    marginacion_gdf, 
    umbral_similitud=75  # Puedes ajustar este valor
)



🚀 Iniciando proceso de Fuzzy Linkage...
=== Iniciando Fuzzy Linkage para COLONIAS ===
🎯 Umbral de similitud: 75%
------------------------------------------------------------
📊 Grupos únicos (Municipio-Entidad) en marginación: 76

🔄 Procesando: df_coordenadas_sustentantes_2020
  📊 Registros en dataset: 25,709
  📋 Asignaciones realizadas: 16,777
  📊 Cobertura: 65.26%
  🎯 Ejemplos de matches (Top 3 por score):
    1. 'nueva san antonio' ↔ 'nueva san antonio'
       Municipio: chalco, Score: 100%
    2. 'caltenco' ↔ 'caltenco'
       Municipio: coyotepec, Score: 100%
    3. 'san pedro atzompa' ↔ 'san pedro atzompa'
       Municipio: tecamac, Score: 100%
  📈 Score promedio: 93.8%
  📉 Score mínimo: 75%
  📊 Score máximo: 100%
  📊 Distribución de scores:
    - 95-100%: 10,647 (63.5%)
    - 90-94%: 1,506 (9.0%)
    - 85-89%: 1,147 (6.8%)
    - 80-84%: 1,609 (9.6%)
    - 75-79%: 1,868 (11.1%)
----------------------------------------

🔄 Procesando: df_coordenadas_sustentantes_2021
  📊 Registros e

: 

In [ ]:
# Verificar resultados usando la función existente
verificar_asignaciones_linkage(original_datasets)

=== Verificación de Asignaciones por Record Linkage ===
------------------------------------------------------------
📊 Dataset: df_coordenadas_sustentantes_2020
  🎯 IM_2020 asignados: 16,777/25,709 (65.26%)
  🏷️  HASH_CP_gpkg asignados: 16,777/25,709 (65.26%)
  📋 Ejemplos de asignaciones:
    - COLONIA: nueva san antonio... | IM_2020: Medio | CP: 56605
    - COLONIA: caltenco... | IM_2020: Medio | CP: 54660
    - COLONIA: san pedro atzompa... | IM_2020: Medio | CP: 55770
----------------------------------------
📊 Dataset: df_coordenadas_sustentantes_2021
  🎯 IM_2020 asignados: 16,355/24,963 (65.52%)
  🏷️  HASH_CP_gpkg asignados: 16,355/24,963 (65.52%)
  📋 Ejemplos de asignaciones:
    - COLONIA: torres de potrero... | IM_2020: Medio | CP: 01840
    - COLONIA: san miguel topilejo... | IM_2020: Alto | CP: 14500
    - COLONIA: barrio san cristobal... | IM_2020: Bajo | CP: 16080
----------------------------------------
📊 Dataset: df_coordenadas_sustentantes_2022
  🎯 IM_2020 asignados: 8,96

: 

In [ ]:
# Ejecutar el record linkage
print("🚀 Iniciando proceso de Record Linkage...")
original_datasets, resultados = realizar_record_linkage_colonias(original_datasets, marginacion_gdf)



🚀 Iniciando proceso de Record Linkage...
=== Iniciando Record Linkage para COLONIAS ===
------------------------------------------------------------
🔄 Procesando: df_coordenadas_sustentantes_2020 (25,709 registros)
  🔍 Pares candidatos: 5,195,954
  🧮 Calculando similitudes...
  ✅ Matches válidos: 73,047
  📋 Asignaciones: 73,047  Cobertura: 284.13%
----------------------------------------
🔄 Procesando: df_coordenadas_sustentantes_2021 (24,963 registros)
  🔍 Pares candidatos: 5,070,329
  🧮 Calculando similitudes...
  ✅ Matches válidos: 72,054
  📋 Asignaciones: 72,054  Cobertura: 288.64%
----------------------------------------
🔄 Procesando: df_coordenadas_sustentantes_2022 (13,796 registros)
  🔍 Pares candidatos: 2,594,224
  🧮 Calculando similitudes...
  ✅ Matches válidos: 39,539
  📋 Asignaciones: 39,539  Cobertura: 286.60%
----------------------------------------
🔄 Procesando: df_coordenadas_sustentantes_2023 (18,693 registros)
  🔍 Pares candidatos: 3,605,709
  🧮 Calculando similitudes.

: 

In [ ]:
# Verificar resultados
verificar_asignaciones_linkage(original_datasets)

=== Verificación de Asignaciones por Record Linkage ===
------------------------------------------------------------
📊 Dataset: df_coordenadas_sustentantes_2020
  🎯 IM_2020 asignados: 20,674/25,709 (80.42%)
  🏷️  HASH_CP_gpkg asignados: 20,674/25,709 (80.42%)
  📋 Ejemplos de asignaciones:
    - COLONIA: ampliacion general jose vicen... | IM_2020: Medio | CP: 57710
    - COLONIA: santa catarina... | IM_2020: Alto | CP: 56033
    - COLONIA: nueva san antonio... | IM_2020: Alto | CP: 56604
----------------------------------------
📊 Dataset: df_coordenadas_sustentantes_2021
  🎯 IM_2020 asignados: 20,193/24,963 (80.89%)
  🏷️  HASH_CP_gpkg asignados: 20,193/24,963 (80.89%)
  📋 Ejemplos de asignaciones:
    - COLONIA: torres de potrero... | IM_2020: Medio | CP: 01840
    - COLONIA: san juan (san pablo oztotepec)... | IM_2020: Medio | CP: 12400
    - COLONIA: san miguel topilejo... | IM_2020: Medio | CP: 14490
----------------------------------------
📊 Dataset: df_coordenadas_sustentantes_2022

: 

In [ ]:
original_datasets["df_coordenadas_sustentantes_2023"]

,COLONIA,CP,MUNICIPIO,ENTIDAD,lat_centro,lng_centro,lat_principal,lng_principal,CP_INDEX,IM_2020,HASH_CP_gpkg,IM_DISTANCE,Distance,HASH_CP_Catalogo,TIPO_ASENTAMIENTO,TIPO_DE_ZONA
0,san andres tetepilco,09440,iztapalapa,cdmx,19.371723,-99.132493,19.373162,-99.130396,09440,Bajo,09850,Medio,0.002334,,,
1,san antonio xahuento,54976,tultepec,edomex,19.690464,-99.112582,19.692443,-99.111031,54976,Alto,54960,Alto,0.002537,,,
2,barrio la asuncion,16040,xochimilco,cdmx,19.271636,-99.101098,19.271636,-99.101098,16040,Medio,16040,Medio,0.002303,,,
3,modelo,57530,nezahualcoyotl,edomex,19.402045,-99.030601,19.402102,-99.030507,57530,Medio,57530,Medio,0.000062,,,
4,hogares marla,55030,ecatepec de morelos,edomex,19.604641,-99.049833,19.604515,-99.049853,55030,Medio,55040,Bajo,0.000057,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18688,alamos,03410,benito juarez,cdmx,19.390471,-99.142494,19.391054,-99.140772,03410,Muy bajo,03400,Muy bajo,0.001309,,,
18689,cobadonga,56607,chalco,edomex,19.275884,-98.910711,19.277908,-98.912117,56607,Alto,56607,Alto,0.001185,,,
18690,gustavo baz prada,54130,tlalnepantla de baz,edomex,19.561345,-99.175959,19.561611,-99.176754,54120,Medio,54120,Medio,0.000563,,,
18691,fraccarbolada los sauces 2,55635,zumpango,edomex,19.815479,-99.042517,19.815731,-99.042252,55635,Medio,55600,Medio,0.002233,,,


: 

In [ ]:
for key, df in original_datasets.items():
    # Contar valores vacíos en IM_2020 (incluyendo strings vacíos)
    valores_vacios = (df['IM_2020'].isnull() | (df['IM_2020'] == '')).sum()

    # Contar total de registros
    total_registros = len(df)

    # Calcular porcentaje de valores vacíos
    porcentaje_vacios = (valores_vacios / total_registros) * 100
    porcentaje_llenos = 100 - porcentaje_vacios
    # Mostrar resultados
    print("-" * 50)
    print(f"Dataset: {key}")
    print(f"Valores vacíos en IM_2020: {valores_vacios}")
    print(f"Total de registros: {total_registros}")
    print(f"Porcentaje de valores vacíos: {porcentaje_vacios:.2f}%")
    print(f"Porcentaje de valores llenos: {porcentaje_llenos:.2f}%")
    print("-" * 50)

    

--------------------------------------------------
Dataset: df_coordenadas_sustentantes_2020
Valores vacíos en IM_2020: 5035
Valores externos (foráneo/extranjero): 560
Total de registros: 25709
Porcentaje de valores vacíos: 20.02%
Porcentaje de valores llenos: 79.98%
--------------------------------------------------
--------------------------------------------------
Dataset: df_coordenadas_sustentantes_2021
Valores vacíos en IM_2020: 4770
Valores externos (foráneo/extranjero): 525
Total de registros: 24963
Porcentaje de valores vacíos: 19.52%
Porcentaje de valores llenos: 80.48%
--------------------------------------------------
--------------------------------------------------
Dataset: df_coordenadas_sustentantes_2022
Valores vacíos en IM_2020: 2699
Valores externos (foráneo/extranjero): 202
Total de registros: 13796
Porcentaje de valores vacíos: 19.85%
Porcentaje de valores llenos: 80.15%
--------------------------------------------------
-------------------------------------------

: 